In [1]:
import findspark 
findspark.init() 

In [2]:
from pyspark import SparkContext, SparkConf 
from pyspark.sql import SparkSession

In [3]:
sc = SparkContext.getOrCreate() 

spark = SparkSession.builder.appName("Python Spark DataFrames basic example").config("spark.some.config.option", 'some-value').getOrCreate() 

In [4]:
employee_df = spark.read.csv("employees.csv", header = True, inferSchema = True)
employee_df.printSchema()

root
 |-- Emp_No: integer (nullable = true)
 |-- Emp_Name: string (nullable = true)
 |-- Salary: integer (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Department: string (nullable = true)



In [5]:
employee_df.show(5)

+------+--------+------+---+----------+
|Emp_No|Emp_Name|Salary|Age|Department|
+------+--------+------+---+----------+
|   198|  Donald|  2600| 29|        IT|
|   199| Douglas|  2600| 34|     Sales|
|   200|Jennifer|  4400| 36| Marketing|
|   201| Michael| 13000| 32|        IT|
|   202|     Pat|  6000| 39|        HR|
+------+--------+------+---+----------+
only showing top 5 rows



In [7]:
employee_df.createTempView('employees')

In [9]:
result = spark.sql('select * from employees where Age > 30')
result.show()

+------+-----------+------+---+----------+
|Emp_No|   Emp_Name|Salary|Age|Department|
+------+-----------+------+---+----------+
|   199|    Douglas|  2600| 34|     Sales|
|   200|   Jennifer|  4400| 36| Marketing|
|   201|    Michael| 13000| 32|        IT|
|   202|        Pat|  6000| 39|        HR|
|   203|      Susan|  6500| 36| Marketing|
|   205|    Shelley| 12008| 33|   Finance|
|   206|    William|  8300| 37|        IT|
|   100|     Steven| 24000| 39|        IT|
|   102|        Lex| 17000| 37| Marketing|
|   103|  Alexander|  9000| 39| Marketing|
|   104|      Bruce|  6000| 38|        IT|
|   105|      David|  4800| 39|        IT|
|   106|      Valli|  4800| 38|     Sales|
|   107|      Diana|  4200| 35|     Sales|
|   109|     Daniel|  9000| 35|        HR|
|   110|       John|  8200| 31| Marketing|
|   111|     Ismael|  7700| 32|        IT|
|   112|Jose Manuel|  7800| 34|        HR|
|   113|       Luis|  6900| 34|     Sales|
|   116|     Shelli|  2900| 37|   Finance|
+------+---

In [11]:
result = spark.sql("select Department, AVG(Salary) as average_salary from employees group by Department")
result.show()

+----------+-----------------+
|Department|   average_salary|
+----------+-----------------+
|     Sales|5492.923076923077|
|        HR|           5837.5|
|   Finance|           5730.8|
| Marketing|6633.333333333333|
|        IT|           7400.0|
+----------+-----------------+



In [12]:
result = spark.sql("select * from employees where Department = 'IT'")
result.show()

+------+--------+------+---+----------+
|Emp_No|Emp_Name|Salary|Age|Department|
+------+--------+------+---+----------+
|   198|  Donald|  2600| 29|        IT|
|   201| Michael| 13000| 32|        IT|
|   206| William|  8300| 37|        IT|
|   100|  Steven| 24000| 39|        IT|
|   104|   Bruce|  6000| 38|        IT|
|   105|   David|  4800| 39|        IT|
|   111|  Ismael|  7700| 32|        IT|
|   129|   Laura|  3300| 38|        IT|
|   132|      TJ|  2100| 34|        IT|
|   136|   Hazel|  2200| 29|        IT|
+------+--------+------+---+----------+



In [14]:
from pyspark.sql.functions import expr

employee_df = employee_df.withColumn("SalaryAfterBonus", expr("Salary * 1.1"))
employee_df.show(5)

+------+--------+------+---+----------+----------------+
|Emp_No|Emp_Name|Salary|Age|Department|SalaryAfterBonus|
+------+--------+------+---+----------+----------------+
|   198|  Donald|  2600| 29|        IT|          2860.0|
|   199| Douglas|  2600| 34|     Sales|          2860.0|
|   200|Jennifer|  4400| 36| Marketing|          4840.0|
|   201| Michael| 13000| 32|        IT|         14300.0|
|   202|     Pat|  6000| 39|        HR|          6600.0|
+------+--------+------+---+----------+----------------+
only showing top 5 rows



In [15]:
result = spark.sql("select Age, Max(Salary) as maximum_salary from employees group by Age")
result.show(5)

+---+--------------+
|Age|maximum_salary|
+---+--------------+
| 31|          8200|
| 34|          7800|
| 28|         12008|
| 27|         17000|
| 26|          3600|
+---+--------------+
only showing top 5 rows



In [17]:
join_df = employee_df.join(employee_df, 'Emp_No', 'inner')
join_df.show(5)

+------+--------+------+---+----------+----------------+--------+------+---+----------+----------------+
|Emp_No|Emp_Name|Salary|Age|Department|SalaryAfterBonus|Emp_Name|Salary|Age|Department|SalaryAfterBonus|
+------+--------+------+---+----------+----------------+--------+------+---+----------+----------------+
|   198|  Donald|  2600| 29|        IT|          2860.0|  Donald|  2600| 29|        IT|          2860.0|
|   199| Douglas|  2600| 34|     Sales|          2860.0| Douglas|  2600| 34|     Sales|          2860.0|
|   200|Jennifer|  4400| 36| Marketing|          4840.0|Jennifer|  4400| 36| Marketing|          4840.0|
|   201| Michael| 13000| 32|        IT|         14300.0| Michael| 13000| 32|        IT|         14300.0|
|   202|     Pat|  6000| 39|        HR|          6600.0|     Pat|  6000| 39|        HR|          6600.0|
+------+--------+------+---+----------+----------------+--------+------+---+----------+----------------+
only showing top 5 rows



In [18]:
from pyspark.sql.functions import avg 

average_age = employee_df.agg(avg("Age")).alias("Average_Age")
average_age.show()

+--------+
|avg(Age)|
+--------+
|   33.56|
+--------+



In [22]:
from pyspark.sql.functions import desc, asc 

employee_df = employee_df.orderBy(asc("Age"), desc("Salary"))
employee_df.show()

+------+---------+------+---+----------+----------------+
|Emp_No| Emp_Name|Salary|Age|Department|SalaryAfterBonus|
+------+---------+------+---+----------+----------------+
|   137|   Renske|  3600| 26| Marketing|          3960.0|
|   101|    Neena| 17000| 27|     Sales|         18700.0|
|   114|      Den| 11000| 27|   Finance|         12100.0|
|   108|    Nancy| 12008| 28|     Sales|         13208.8|
|   130|    Mozhe|  2800| 28| Marketing|          3080.0|
|   126|    Irene|  2700| 28|        HR|          2970.0|
|   204|  Hermann| 10000| 29|   Finance|         11000.0|
|   115|Alexander|  3100| 29|   Finance|          3410.0|
|   134|  Michael|  2900| 29|     Sales|          3190.0|
|   198|   Donald|  2600| 29|        IT|          2860.0|
|   140|   Joshua|  2500| 29|   Finance|          2750.0|
|   136|    Hazel|  2200| 29|        IT|          2420.0|
|   120|  Matthew|  8000| 30|        HR|          8800.0|
|   110|     John|  8200| 31| Marketing|          9020.0|
|   127|    Ja

In [23]:
from pyspark.sql.functions import count 

employ_count = employee_df.groupBy("Department").agg(count("Emp_No")).alias('Number_Employee')
employ_count.show()

+----------+-------------+
|Department|count(Emp_No)|
+----------+-------------+
|     Sales|           13|
|        HR|            8|
|   Finance|           10|
| Marketing|            9|
|        IT|           10|
+----------+-------------+



In [26]:
result = spark.sql("select * from employees where Emp_Name like '%o%'")
result.show()

+------+-----------+------+---+----------+
|Emp_No|   Emp_Name|Salary|Age|Department|
+------+-----------+------+---+----------+
|   198|     Donald|  2600| 29|        IT|
|   199|    Douglas|  2600| 34|     Sales|
|   110|       John|  8200| 31| Marketing|
|   112|Jose Manuel|  7800| 34|        HR|
|   130|      Mozhe|  2800| 28| Marketing|
|   133|      Jason|  3300| 38|     Sales|
|   139|       John|  2700| 36|     Sales|
|   140|     Joshua|  2500| 29|   Finance|
+------+-----------+------+---+----------+



In [27]:
spark.stop()